In [2]:
from credentials import login
from influxdb import InfluxDBClient
import influxdb
import pandas as pd


In [3]:
dfClient = InfluxDBClient(host="testresults.qt.io", port=443, ssl=True,
            verify_ssl=True,username=login["user"], password=login["password"], database='coin')

dfClient._InfluxDBClient__baseurl = "{0}://{1}:{2}/{3}".format(
            dfClient._scheme,
            dfClient._host,
            dfClient._port,
            "influxdb")

In [58]:
def execQuery(query):
    queryResult = dfClient.query(query, database="coin")
    resultDF = pd.DataFrame(queryResult.get_points())
    try:        
        resultDF["testCaseFunction"] = resultDF["testCase"] + resultDF["testFunction"]    
        #print("Non unique: ", resultDF.shape[0])
        resultDF.drop_duplicates(subset ="testCaseFunction", keep = False, inplace = True)
         #print("unique: ", resultDF.shape[0])            
        howMany = resultDF.shape[0]
    except KeyError:
    #       print('KeyError')            
        howMany = 0
    return howMany


In [62]:

from datetime import datetime
from calendar import monthrange

df = pd.DataFrame({
    'flaky_count': [],
    'flaky_failed_count': [],
    'crashed_count': [],
    'date': []
})

today = datetime.now()
print(today)
for year in range (2019, 2021):
    for month in range(1,13):    
        firstDay = datetime(year, month, 1, 0,0,0)
        lastDay = datetime(year, month, monthrange(year,month)[1],23, 59, 59 )       
        
        query = "select * from flaky_test where time > '"+ str(firstDay) + "' and time < '" + str(lastDay) + "'"
        howManyFlaky = execQuery(query)            
        query = "select * from flaky_test where result = 'Failed' and time > '"+ str(firstDay) + "' and time < '" + str(lastDay) + "'"
        howManyFlakyFailed = execQuery(query)            
        query = "select * from crashed_test where time > '"+ str(firstDay) + "' and time < '" + str(lastDay) + "'"        
        howManyCrashed = execQuery(query)
        df = df.append({'flaky_count': howManyFlaky,'flaky_failed_count': howManyFlakyFailed, 'crashed_count': howManyCrashed, 'date': firstDay}, ignore_index=True)
        if today < lastDay:            
            break



2020-03-10 00:13:20.081454


In [77]:
from bokeh.plotting import figure
from bokeh.io import output_file, show
from bokeh.models import DatetimeTickFormatter
from bokeh.transform import dodge
output_file("distinct_flaky_and_crashed.html")

src = ColumnDataSource(df)

ms_factor = 1000000000 # i have to multiply by 1000 000 000 since date is presented up to ms
bar_width = 0.4 * ms_factor 

hover_tool = HoverTool( tooltips=[('flaky', '@flaky_count'),('flaky failed', '@flaky_failed_count'),('crashed', '@crashed_count'), ('date', '@date{%F}')], formatters={'date': 'datetime'})


p = figure(x_axis_type="datetime", plot_width=800, plot_height=400, title="Distinct flaky testcases per month", tools=[hover_tool], toolbar_location="below")
p.vbar(x=dodge('date',  -0.5 * ms_factor ,), top='flaky_count',source=ColumnDataSource(df), width= bar_width, color="#c9d9d3", legend_label="Flaky")
p.vbar(x=dodge('date',  0.0,) , top='flaky_failed_count',source=ColumnDataSource(df), width=bar_width, color="#718dbf", legend_label="Flaky_Failed")
p.vbar(x=dodge('date',  0.5 * ms_factor,) , top='crashed_count',source=ColumnDataSource(df), width=bar_width, color="#e84d60", legend_label="Crashed")

p.y_range.start = 0
p.legend.location = "top_left"
p.legend.orientation = "horizontal"

show(p)



1.4.0
